# Seismology

Continuing on our theme of elasticity, we're going to look at seismic wave propagation in earth's mantle.
This is a time-dependent wave propagation problem and we're going to need to do some mesh generation.
Once again, if $u$ is the displacement of the mantle, and $v = \dot u$ is the velocity, the extensive quantity is the momentum $\rho v$ and the flux is the stress $\sigma$.
The constitutive law is
$$\sigma = 2\mu\,\varepsilon + \lambda\,\text{tr}(\varepsilon)I$$
where $\varepsilon = (\nabla u + \nabla u^\top)/2$ is the infinitesimal strain tensor and $\mu$, $\lambda$ are the Lamé parameters.

Note that if you contract together a symmetric and an anti-symmetric tensor, you get zero.
This is going to let us rewrite some parts of the variational form.
If we consider a test displacement $w$, then we can write
$$\nabla w = \frac{1}{2}\left(\nabla w + \nabla w^\top\right) + \frac{1}{2}\left(\nabla w - \nabla w^\top\right),$$
i.e. as the sum of a symmetric and an anti-symmetric part.
Now if we contract the strain tensor of $u$ with $\nabla w$, we'll wipe out the anti-symmetric part of $\nabla w.

All this is to say that the weak form of the seismic wave equation is
$$\int_\Omega\left(\rho\ddot u\cdot w - 2\mu\,\varepsilon(u):\varepsilon(w) - \lambda\;\text{tr}\,\varepsilon(u)\;\text{tr}\,\varepsilon(w) - f\cdot w\right)dx = 0$$
for any test displacement $w$.
Don't grow too attached to this form, we're about to not use it at all.

Like the shallow water wave problem that I showed before, seismic wave propagation is second-order in time.
We'll thus have to split it into a system of first-order equations.
Unlike the shallow water problem that I showed before, we're going to carry out this splitting in a slightly more interesting way.
Rather than work with the displacement and velocity, we can instead take a time-derivative of the linear elastic constitutive law to write
$$\dot\sigma = 2\mu\,\dot\varepsilon + \lambda\,\text{tr}(\dot\varepsilon)I$$
As we'll see, this decomposition of the problem will have certain advantages.

### But first, meshing

To do all this, we're going to first have to make a triangular mesh of our geometry.
There are lots of mesh generators, many of which are unfortunately academic abandonware.
We're going to use [Triangle](https://www.cs.cmu.edu/~quake/triangle.html), which was the first good open-source tool for 2D meshing.
It hasn't been modified since 2005, but it's got good documentation and the file formats make sense.
There's a Python wrapper for it called [MeshPy](https://documen.tician.de/meshpy/) which we'll use here.

MeshPy has a few convenience functions for defining simple geometries like boxes and circles.
These are in the namespace `meshpy.geometry`.
We'll start by just making a box.
The procedure is to create this `GeometryBuilder` object, and then add more things to it.
Here we're only adding one, but I could also use this to create shapes with holes in them.

In [ ]:
import meshpy, meshpy.geometry, meshpy.triangle

builder = meshpy.geometry.GeometryBuilder()

lx = 200e3
ly = 100e3
box = meshpy.geometry.make_box((0.0, 0.0), (lx, ly))
builder.add_geometry(*box)

Now we'll actually create the mesh.
The utility of this GeometryBuilder is that it will internally keep track of what indices everything should be assigned and what boundary markers they should get.
This is all very annoying so having someone do it for you is a huge time saver.
The process is to first create a data structure (`MeshInfo`) that describes the boundary of the domain we're interested in.
The geometry builder will fill in everything that needs to go in this MeshInfo.
Then we can call `meshpy.triangle.build` to actually triangulate the domain.
Here I'm using a resolution of 4km, which as we'll see is very coarse.

In [ ]:
mesh_info = meshpy.triangle.MeshInfo()
builder.set(mesh_info)

length = 4e3
triangle_mesh = meshpy.triangle.build(mesh_info, max_volume=0.5 * length**2)

Just so we can see the result, I'm pulling out all the points and triangles of the mesh.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

points = np.array(triangle_mesh.points)
triangles = np.array(triangle_mesh.elements)

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.triplot(points[:, 0], points[:, 1], triangles);

The code below will convert MeshPy's data structure to Firedrake.
Don't ask how I figured this out.

In [ ]:
import firedrake
from firedrake.cython import dmcommon

def triangle_to_firedrake(triangle_mesh):
    points = np.array(triangle_mesh.points)
    triangles = np.array(triangle_mesh.elements)
    plex = firedrake.mesh.plex_from_cell_list(2, triangles, points, firedrake.COMM_WORLD)
    markers = {
        tuple(sorted((v1, v2))): triangle_mesh.facet_markers[index]
        for index, (v1, v2) in enumerate(triangle_mesh.facets)
    }
    plex.createLabel(dmcommon.FACE_SETS_LABEL)
    plex.markBoundaryFaces("boundary_faces")
    boundary_faces = plex.getStratumIS("boundary_faces", 1).getIndices()
    offset = plex.getDepthStratum(0)[0]
    for face in boundary_faces:
        vertices = tuple(sorted([v - offset for v in plex.getCone(face)]))
        marker = markers[vertices]
        plex.setLabelValue(dmcommon.FACE_SETS_LABEL, face, marker)
    
    return firedrake.Mesh(plex, reorder=False)

Now we'll plot the Firedrake mesh.
I'm showing this mostly so you can see that we got all the boundary markers.

In [ ]:
mesh = triangle_to_firedrake(triangle_mesh)

fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(mesh, axes=ax)
ax.legend();

### The simulation

Now we get to the actual physics.
I wrote down a second-order wave equation for the displacements above.
In order to use our usual ODE solvers, we need to reduce this to a 1st-order problem.
We're going to do that by using both the velocity $v = \dot u$ and the stress tensor as unknowns.
We know that the rate of change of the stress tensor can be expressed in terms of the strain *rate* tensor
$$\varepsilon(v) = \frac{1}{2}\left(\nabla v + \nabla v^\top\right) = \frac{1}{2}\left(\nabla\dot u + \nabla \dot u^\top\right).$$
So we can write the evolution equation for $\sigma$ as
$$\dot \sigma = 2\mu\varepsilon(v) + \lambda\,\text{tr}\,\varepsilon(v)\,I$$
and so the variational form is
$$\int_\Omega\left\{\rho\dot v\cdot w + \sigma\cdot\varepsilon(w) - f\cdot w + \dot\sigma\cdot\tau - 2\mu\varepsilon(v):\tau - \lambda\,\text{tr}\,\varepsilon(v)\,\text{tr}(\tau)\right\}dx = 0$$
for all test velocities $w$ and stresses $\tau$.
There are no boundary terms because I'm assuming the external stresses are zero (more on this in a moment).

First I'll have to create some physical constants.
I'm assuming a shear wave speed of 7 km/s, a compressional wave speed of 11 km/s, and a density of 4500 kg/m${}^3$.
We can then compute the parameters $\mu$ and $\lambda$ from the wave speeds and material density.

In [ ]:
s_wave_speed = 7e3
p_wave_speed = 11e3
density = 4.5e3
lame_μ = density * s_wave_speed**2
lame_λ = density * (p_wave_speed**2 - 2 * s_wave_speed**2)

Once again, we'll create finite elements for the velocity and stress paces.
Here we're using tensor function spaces for the first time.
We added an extra argument `symmetry=True` in order to guarantee that we get only symmetric tensors.

In [ ]:
cg_1 = firedrake.FiniteElement("CG", "triangle", 1)
dg_0 = firedrake.FiniteElement("DG", "triangle", 0)

V_1 = firedrake.VectorFunctionSpace(mesh, cg_1)
Σ_0 = firedrake.TensorFunctionSpace(mesh, dg_0, symmetry=True)

Z_1 = V_1 * Σ_0

Now we have to talk about seismic sources and I'm going to reveal my terrible ignorance of seismology.
The internet and the book by Aki tell me that a reasonable seismic source is
$$f_i(x, t) = -M_{ij}(t)\partial_j\delta(x - x_s)$$
where $M_{ij}$ is the seismic *moment tensor* as a function of time, $x_s$ is the earthquake source location, and $\delta$ is the [delta function](https://en.wikipedia.org/wiki/Dirac_delta_function).
If you're unfamiliar with the delta function, you can think of it as a normal distribution with a very small variance.
Here I'll take the souce term to be 600 km down from the surface at the very top of the domain.
This is a fairly deep earthquake.

The time dependence of a typical moment tensor is
$$M_{ij}(t) = M_{ij}^0R(t)$$
where $R$ is a [Ricker wavelet](https://en.wikipedia.org/wiki/Ricker_wavelet):
$$R(t) = \left(1 - 2\pi^2\frac{t^2}{T^2}\right)\exp\left(-\pi^2\frac{t^2}{T^2}\right)$$
Here we'll take $T$ to be 0.25s; I've seen other references mention things on the order of 0.1s.

I have no idea what a reasonable value for the moment tensor is.
Luckily, this problem is linear, so if you change the scale of the moment tensor, you'll get a proportional change in the scale of the stress and velocity fields.

In [ ]:
from firedrake import Constant, inner, outer, sym, tr, grad, dx, ds as dγ
from numpy import pi as π

ρ = Constant(density)
μ = Constant(lame_μ)
λ = Constant(lame_λ)

t = Constant(0.0)
t_0 = Constant(0.5)
T = Constant(0.25)

R = (1 - 2 * π**2 * (t - t_0)**2 / T**2) * firedrake.exp(-π**2 * (t - t_0)**2 / T**2)
fault_length = 100e3
fault_height = 1e3
slip = 1.0
m = Constant(μ * fault_length * fault_height * slip)
M_0 = m * firedrake.as_tensor([[1.0, -1.0], [-1.0, 1.0]])

To finish the definition of the sources, we need to define the delta function or something close enough to it.
Here we'll use a Gaussian, normalized so that it integrates to 1.
This is a bit of a dirty hack but it'll work for a demonstration.
It is possible to include real honest delta functions in Firedrake if you need to.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
depth = 30e3
x_s = Constant((lx / 2, ly - depth))
σ = Constant(5e3)
expr = firedrake.exp(-inner(x - x_s, x - x_s) / (2 * σ**2))
norm = firedrake.assemble(expr * dx)
δ = expr / norm

In [ ]:
Q = firedrake.FunctionSpace(mesh, "DG", 0)
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(firedrake.Function(Q).project(δ), axes=ax)
fig.colorbar(colors);

Finally we define the source term for our problem as the product of the time-dependent part $R$, the magnitude $M_0$, and a delta function.

In [ ]:
M = R * M_0 * δ

Some helper functions that should be familiar from the Stokes demo.

In [ ]:
def ε(u):
    return sym(grad(u))

I = firedrake.Identity(2)

And finally the definition of the variational form from before.
I'm wrapping up hte definition of the variational form in a Python function because we're going to solve this same problem more than once.
Here I've split the definition of the form itself into two parts, one for the velocities and one for the stresses, that are added together.

In [ ]:
import irksome
from irksome import Dt

z = firedrake.Function(Z_1)

def variational_form(z, M):
    v, σ = firedrake.split(z)
    w, τ = firedrake.TestFunctions(z.function_space())
    F_v = (ρ * inner(Dt(v), w) + inner(σ, ε(w)) - inner(M, ε(w))) * dx
    F_σ = (inner(Dt(σ), τ) - 2 * μ * inner(ε(v), τ) - λ * tr(ε(v)) * tr(τ)) * dx
    return F_v + F_σ

The rest of the simulation is a standard use of Irksome.
The main difference here is the time-dependent source  term.
Note that here I have to keep updating the variable `t` so that our source term actually does change in time.

In [ ]:
dt = Constant(0.01)
method = irksome.GaussLegendre(1)
solver = irksome.TimeStepper(variational_form(z, M), method, t, dt, z)

In [ ]:
import tqdm

zs_coarse = [z.copy(deepcopy=True)]
final_time = 10.0
num_steps = int(final_time / float(dt))
for step in tqdm.trange(num_steps):
    solver.advance()
    t.assign(t + dt)
    zs_coarse.append(z.copy(deepcopy=True))

In [ ]:
v, σ = z.subfunctions

Something at least vaguely sensible is happening (waves are propagating!) but we can obviously see the imprint of the mesh.
Can we do better?

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(zs_coarse[0].sub(0).sub(1), vmin=-30.0, vmax=30.0, cmap="RdBu", num_sample_points=4, axes=ax)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(z):
    colors.set_array(fn_plotter(z.sub(0).sub(1)))

animation = FuncAnimation(fig, animate, zs_coarse[::2], interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())

### Simulation, again

The first thing we might try is running the simulation again, but using higher-order elements.
Here the advantage of wrapping up the creation of the variational form into a function that we can reuse becomes really clear.

In [ ]:
cg_2 = firedrake.FiniteElement("CG", "triangle", 2)
dg_1 = firedrake.FiniteElement("DG", "triangle", 1)

V_2 = firedrake.VectorFunctionSpace(mesh, cg_2)
Σ_1 = firedrake.TensorFunctionSpace(mesh, dg_1, symmetry=True)

Z_2 = V_2 * Σ_1

z = firedrake.Function(Z_2)

t.assign(0.0)
solver = irksome.TimeStepper(variational_form(z, M), method, t, dt, z)

In [ ]:
zs_fine = [z.copy(deepcopy=True)]
for step in tqdm.trange(num_steps):
    solver.advance()
    t.assign(t + dt)
    zs_fine.append(z.copy(deepcopy=True))

Much nicer looking this time around.
Can we do even better?

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(zs_fine[0].sub(0).sub(1), vmin=-30.0, vmax=30.0, cmap="RdBu", num_sample_points=4, axes=ax)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(z):
    colors.set_array(fn_plotter(z.sub(0).sub(1)))

animation = FuncAnimation(fig, animate, zs_fine[::2], interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())

### Error estimation

The key idea behind a posteriori error estimation is **using the discrepancy between lower- and higher-order solutions**.
Where they differ most tells you something important.
The code below creates a variable `discrepancy` which takes constant values in every triangle.
The loop goes through the coarse and fine solutions at every timestep, computes their RMS difference, and adds it to the total time-integrated discrepancy.
We're only using this discrepancy field to decide how we'll refine our mesh, so we don't need to normalize it.

In [ ]:
discrepancy = firedrake.Function(Q)
for z_coarse, z_fine in zip(zs_coarse, zs_fine):
    v_c, v_f = z_coarse.sub(0), z_fine.sub(0)
    δv = v_f - v_c
    discrepancy.project(discrepancy + firedrake.sqrt(inner(δv, δv)))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(discrepancy, axes=ax);
#fig.colorbar(colors);

I'm showing you only one way particularly blunt way to estimate the error, but you should know that there are loads more.
For example, we could be looking at discrepancies between the stresses instead of the velocities; we could be using a different norm; and there are even error estimation strategies that don't require two different discretizations.

### Mesh refinement

So we've made some field that tells us about the total error.
How should we use this information?
First, we'll want to compute the area of every triangle of the mesh.
When we go to refine the mesh, we'll modify this `areas` field in place to define our desired triangle areas.

In [ ]:
areas = firedrake.project(firedrake.CellVolume(mesh), Q)
print(f"Min/max triangle area: {areas.dat.data_ro.min()/1e6:.1f}, {areas.dat.data_ro.max()/1e6:.1f} km²")

Next, I'm extracting the raw data out of the `discrepancy` field and using the numpy function `argsort` to return a permutation that sorts it.
If I then take the tail of this `order` array, it tells me the indices of all the triangles that have the largest discrepancies.
Finally, I'm modifying the `areas` array in place along those most offending indices to reduce their value by 4.

In [ ]:
order = np.argsort(discrepancy.dat.data_ro)
num_triangles = mesh.num_cells()
indices = order[-num_triangles // 4:]
areas.dat.data[indices] /= 4.0

Now we go back to our mesh generator.
First, we tell the mesh that we want to allocate space for an array of desired triangle areas.
We can then fill in the array of desired element volumes with the values that we just calculated above.

In [ ]:
triangle_mesh.element_volumes.setup()
for index in range(num_triangles):
    triangle_mesh.element_volumes[index] = areas.dat.data_ro[index]

Now we'll call out to the mesh generator and ask it to refine the triangle mesh.

In [ ]:
refined_triangle_mesh = meshpy.triangle.refine(triangle_mesh)

We can see from the plot below that we get much more density in some regions in space.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.triplot(*np.array(refined_triangle_mesh.points).T, np.array(refined_triangle_mesh.elements));

And we'll once again transform the data structure from MeshPy into a Firedrake mesh.

In [ ]:
refined_mesh = triangle_to_firedrake(refined_triangle_mesh)

Now that we're working on a new, refined mesh, we have to recreate the source function to be defined on it.

In [ ]:
x = firedrake.SpatialCoordinate(refined_mesh)
depth = 30e3
x_s = Constant((lx / 2, ly - depth))
σ = Constant(5e3)
expr = firedrake.exp(-inner(x - x_s, x - x_s) / (2 * σ**2))
norm = firedrake.assemble(expr * dx)
δ = expr / norm

We have to remember to re-set the time variable back to 0.

In [ ]:
t.assign(0.0)
M = R * M_0 * δ

Now we'll run the simulation again at two different discretizations.


In [ ]:
V_1 = firedrake.VectorFunctionSpace(refined_mesh, cg_1)
Σ_0 = firedrake.TensorFunctionSpace(refined_mesh, dg_0, symmetry=True)
Z_1 = V_1 * Σ_0

z_1 = firedrake.Function(Z_1)

V_2 = firedrake.VectorFunctionSpace(refined_mesh, cg_2)
Σ_1 = firedrake.TensorFunctionSpace(refined_mesh, dg_1, symmetry=True)
Z_2 = V_2 * Σ_1

z_2 = firedrake.Function(Z_2)

In [ ]:
solver_1 = irksome.TimeStepper(variational_form(z_1, M), method, t, dt, z_1)
solver_2 = irksome.TimeStepper(variational_form(z_2, M), method, t, dt, z_2)

In [ ]:
zs_1 = [z_1.copy(deepcopy=True)]
zs_2 = [z_2.copy(deepcopy=True)]

for step in tqdm.trange(num_steps):
    solver_1.advance()
    solver_2.advance()
    t.assign(t + dt)
    zs_1.append(z_1.copy(deepcopy=True))
    zs_2.append(z_2.copy(deepcopy=True))

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(zs_2[0].sub(0).sub(1), vmin=-30.0, vmax=30.0, cmap="RdBu", num_sample_points=4, axes=ax)

fn_plotter = firedrake.FunctionPlotter(refined_mesh, num_sample_points=4)
def animate(z):
    colors.set_array(fn_plotter(z.sub(0).sub(1)))

animation = FuncAnimation(fig, animate, zs_2[::2], interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())

### What now

* Use an actually sensible seismic source instead of whatever I made up.
* Change the S- and P-wave speeds in parts of the domain to be something more realistic, see [this](https://opentextbc.ca/physicalgeology2ed/chapter/9-1-understanding-earth-through-seismology/) for example.
* On that note, you could also use a standard test case like the Marmousi velocity model.
* Make the elasticity tensor anisotropic.
* Use a larger spatial domain where the curvature of the earth can't be ignored.
* Try this in 3D. MeshPy has an interface to TetGen.
* Viscoelasticity -- change the constitutive relation to $\dot\sigma + t_m^{-1}\sigma = \mathscr{C}\varepsilon$.
* Try different ways of estimating the error; try changing the triangle distribution in the middle of the simulation and reprojecting.